In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.graph_objects as go

# U.S. STOCK MARKET RETURN

In [ ]:
mkt = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]/100
mkt = mkt['Mkt-RF'] + mkt.RF
mkt.index.name = "year"
mkt.name = "market"
mkt.index = mkt.index.astype(str)
mkt.tail()

# RETURNS FROM YAHOO FINANCE

In [ ]:
ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]
price = price.resample("Y").last()
ret = price.pct_change()[:-1].dropna()
ret.index = ret.index.map(lambda x: x.year)
ret.index.name = "year"
ret.name = ticker
ret.tail()

# YAHOO RETURN OR MARKET

In [ ]:
flag = int(input("input 0 for market or 1 for yahoo return"))
data = ret if flag else mkt
name = f"{ticker.upper()}" if flag else "Market"

# BEST AND WORST PERIODS

In [ ]:
years = int(input("input desired length of time period in years "))

compound = data.rolling(years).apply(lambda x: (1 + x).prod())
bestyear = compound.idxmax()
worstyear = compound.idxmin()

geomean = compound.dropna()**(1/years) - 1
geomean = geomean.reset_index()
geomean.columns = ["endyear", "geomean"]
geomean["startyear"] = compound.endyear - years + 1

best = data.loc[(bestyear - years + 1) : bestyear]
best = (1 + best).cumprod()
best = best.reset_index()
best.columns = ["year", "compounded_return"]
best["startyear"] = best.year.iloc[0]

worst = data.loc[(worstyear - years + 1) : bestyear]
worst = (1 + worst).cumprod()
worst = worst.reset_index()
worst.columns = ["year", "compounded_return"]
worst["startyear"] = worst.year.iloc[0]

# FIGURE 1

In [ ]:
trace = go.Scatter(
    x=best.year,
    y=best.compounded_return,
    text=best.startyear,
    hovertemplate="%{text}-%{x}<br>$%{y:.2f}",
    mode="lines+markers"
)
fig = go.Figure(trace)
fig.update_layout(
    title = f"Best {years} Year Period"
    xaxis_title="Year",
    yaxis_title=f"{name} Accumulation from $1",
    yaxis_tickprefix="$",
    yaxis_tickformat=".2f",
    template="plotly_white"
)
fig.show()

# FIGURE 2

In [ ]:
trace = go.Scatter(
    x=worst.year,
    y=worst.compounded_return,
    text=worst.startyear,
    hovertemplate="%{text}-%{x}<br>$%{y:.2f}",
    mode="lines+markers"
)
fig = go.Figure(trace)
fig.update_layout(
    title = f"Worst {years} Year Period"
    xaxis_title="Year",
    yaxis_title=f"{name} Accumulation from $1",
    yaxis_tickprefix="$",
    yaxis_tickformat=".2f",
    template="plotly_white"
)
fig.show()

# FIGURE 3

In [ ]:
trace = go.Scatter(
    x=geomean.year,
    y=geomean.geomean,
    text=geomean.startyear,
    hovertemplate="%{text}-%{x}<br>%{y:.2%}",
    mode="lines+markers"
)
fig = go.Figure(trace)
fig.update_layout(
    title = f"{years} Year {name} Geometric Average Returns"
    xaxis_title="Ending Year",
    yaxis_title="Geometric Average Return",
    yaxis_tickformat=".2f",
    template="plotly_white"
)
fig.show()

# FIGURE 4

In [ ]:
trace = go.Box(
    x = geomean.geomean,
)
fig = go.Figure(trace)
fig.update_layout(
    template="plotly_white"
)
fig.show()
